# ETL For Project 4 - Fake News Detection

Import Dependencies

In [1]:
import os
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import inspect

Set Path For CSV Files

In [10]:
csv_fake = os.path.join('..', 'Dataset', 'test.csv')
csv_true = os.path.join('..', 'Dataset', 'train.csv')

Load CSV Files

In [11]:
fake_df = pd.read_csv(csv_fake)
fake_df.head()

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...


In [12]:
true_df = pd.read_csv(csv_true)
true_df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


# !!!!!!!!!!!!!!!!!!!!!!!!!!!!! Need to clean dirty rows from fake.csv before convert

Convert Date Column Data Type

In [13]:
true_df = true_df.astype({'date':'datetime64[ns]'})

KeyError: 'Only a column name can be used for the key in a dtype mappings argument.'

In [6]:
fake_df = fake_df.astype({'date':'datetime64[ns]'})

NameError: name 'fake_df' is not defined

In [7]:
true_df.head()

NameError: name 'true_df' is not defined